####  Run this cell to set up and start your interactive session.


####  s3://deupskilling-weatherforecast/LandingZone/current_data.parquet/


In [0]:
%idle_timeout 2880
%glue_version 4.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.5 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 35e3c84d-7c22-4553-be38-bcc3a1f2ad21
Applying the following default arguments:
--glue_kernel_version 1.0.5
--enable-glue-datacatalog true
Waiting for session 35e3c84d-7c22-4553-be38-bcc3a1f2ad21 to get into ready status...
Session 35e3c84d-7c22-4553-be38-bcc3a1f2ad21 ha

In [0]:
print('hello')

hello


In [0]:
# listing all the parquet files
import boto3

s3 = boto3.client('s3')
objects = s3.list_objects_v2(Bucket='deupskilling-weatherforecast', Prefix='LandingZone/', Delimiter='/')

for obj in objects.get('Contents', []):
    print(obj['Key'])

folders = [content['Prefix'] for content in objects.get('CommonPrefixes', [])]
for folder in folders:
    print(folder)


LandingZone/7day_hourly_forecast.parquet/
LandingZone/cities_dim.parquet/
LandingZone/current_data.parquet/
LandingZone/daily_historical.parquet/
LandingZone/daily_historical_forecast.parquet/
LandingZone/hourly_historical.parquet/
LandingZone/hourly_historical_forecast.parquet/
LandingZone/test.parquet/


In [0]:
files = dbutils.fs.ls(parquet_dir)

# Filter out the Parquet files (if the folder contains files of other types too)
parquet_files = [file.path for file in files if file.path.endswith('.parquet')]

# Print all Parquet files
for parquet_file in parquet_files:
    print(parquet_file)

# If you want to list folders (similar to the 'CommonPrefixes' in S3)
# Folders are directories in Databricks, and they will have a trailing '/'
folders = [file.path for file in files if file.isDir()]

# Print all folder paths
for folder in folders:
    print(folder)

In [0]:
files = dbutils.fs.ls("/mnt/de-upskilling-weather/LandingZone/")

parquet_files = [file.path for file in files if file.path.endswith('.parquet/')]

# Print all folder paths
for file in parquet_files:
    print(file)

#dbutils.fs.ls("/mnt/de-upskill-weatherforecasting/LandingZone/")

dbfs:/mnt/de-upskilling-weather/LandingZone/7day_daily_forecast.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/7day_hourly_forecast.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/cities_dim.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/current_data.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/daily_historical.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/daily_historical_forecast.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/hourly_historical.parquet/
dbfs:/mnt/de-upskilling-weather/LandingZone/hourly_historical_forecast.parquet/


In [0]:
seven_day_daily = spark.read.parquet('s3://deupskilling-weatherforecast/LandingZone/7day_daily_forecast.parquet')

In [0]:
seven_day_daily = seven_day_daily.withColumnRenamed('latitute', 'latitude')

In [0]:
seven_day_daily.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- timezone: string (nullable = true)
 |-- weather_code: double (nullable = true)
 |-- temperature_2m_max: double (nullable = true)
 |-- temperature_2m_min: double (nullable = true)
 |-- apparent_temperature_max: double (nullable = true)
 |-- apparent_temperature_min: double (nullable = true)
 |-- precipitation_sum: double (nullable = true)
 |-- rain_sum: double (nullable = true)
 |-- showers_sum: double (nullable = true)
 |-- snowfall_sum: double (nullable = true)
 |-- wind_speed_10m_max: double (nullable = true)
 |-- wind_gusts_10m_max: double (nullable = true)
 |-- wind_direction_10m_dominant: double (nullable = true)


In [0]:
seven_day_daily.show(5)

Py4JJavaError: An error occurred while calling o87.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 4 times, most recent failure: Lost task 0.3 in stage 8.0 (TID 171) (172.34.52.60 executor 4): org.apache.spark.sql.execution.datasources.FileDownloadException: Failed to download file path: s3://deupskilling-weatherforecast/LandingZone/7day_daily_forecast.parquet/part-00000-136f62d9-aecf-4f04-a652-cbc05b6f6512-c000.snappy.parquet, range: 0-1772, partition values: [empty row], isDataPresent: false, eTag: e0fdc462ac3b5baac799d04230055b2c-1
	at org.apache.spark.sql.execution.datasources.AsyncFileDownloader.next(AsyncFileDownloader.scala:142)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.getNextFile(FileScanRDD.scala:423)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:337)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scal

In [0]:
df = spark.read.parquet('s3://deupskilling-weatherforecast/LandingZone/test.parquet')
df

DataFrame[date: timestamp, latitute: double, longitude: double, temperature_2m_max: double, temperature_2m_min: double, temperature_2m_mean: double, apparent_temperature_max: double, apparent_temperature_min: double, apparent_temperature_mean: double, precipitation_sum: double, rain_sum: double, snowfall_sum: double, wind_speed_10m_max: double, wind_gusts_10m_max: double, wind_direction_10m_dominant: double]


In [0]:
df = df.withColumnRenamed('latitute', 'latitude')

In [0]:
df.write.mode('overwrite').parquet('s3://deupskilling-weatherforecast/LandingZone/test.parquet')

Py4JJavaError: An error occurred while calling o136.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:279)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:193)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:103)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker$.withTracker(QueryPlanningTracker.scala:107)
	at org.a